In [21]:
cd ..

/Users/johanbarreiro/Documents/GitHub


In [22]:
import pandas as pd
from pyomo.environ import NonNegativeReals, NonNegativeIntegers
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, Set, Param, NonNegativeReals, minimize

Read the parameter data:

In [23]:
# Read the CSV file into a DataFrame
df = pd.read_csv('optimizer/optimizer_input/parameters.csv')

df = df.dropna()

# Choose the 24-hour period you want to optimize (for example, starting on 2024-07-01 00:00:00)
start_date = '2023-06-01 00:00:00'
end_date = '2023-06-02 00:00:00'

# Filter the data for the chosen 24-hour period
data = df[(df['Time'] >= start_date) & (df['Time'] < end_date)]

# Ensure that the data is sorted by timestamp and reset the index
data = data.sort_values(by='Time').reset_index(drop=True)

# Check the filtered data
data


FileNotFoundError: [Errno 2] No such file or directory: 'optimizer/optimizer_input/parameters.csv'

Create a model:

In [ ]:
# Define the model
model = ConcreteModel(name="Daily_Price_Optimizer")

# Display the model
print(model)

**Define Sets:** 

In [ ]:
# Define the set of hours
model.hours = Set(initialize=range(0, 24))

**Define Parameters:**
Parameters are values that are given to us, and we don’t have control over them.

In [ ]:
# Define the parameters and initialize them with data from the CSV
model.production_schedule = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['production_schedule'].to_dict())

model.maintenance_status = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['maintenance_status'].to_dict())

model.volume_production_waste = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['volume_production_waste'].to_dict())

model.number_of_workers = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['number_of_workers'].to_dict())

model.heat_index = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['heat_index'].to_dict())

model.efficiency_adjusted_power = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['efficiency_adjusted_power'].to_dict())

model.compressors_energy = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['compressor_consumption'].to_dict())

model.operational_presence = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['operational_presence'].to_dict())

model.fabric_in_chamber = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['fabric_in_chamber'].to_dict())

model.testing_schedule = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['testing_schedule'].to_dict())

model.workload = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['workload'].to_dict())

model.technological_centers_energy = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['technological_centers_consumption'].to_dict())

model.lightbulbs_active = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['lightbulbs_active'].to_dict())

model.active_wall_plugs = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['active_wall_plugs'].to_dict())

model.active_computers = Param(model.hours, within=NonNegativeIntegers, initialize=data.set_index('Hour')['active_computers'].to_dict())

model.price_mWh = Param(model.hours, within=NonNegativeReals, initialize=data.set_index('Hour')['price_mWh'].to_dict())

# Display the model
print(model)

**Define Decision Variables with bounds:**
This means  x  and  y  are the variables we want to determine, and they must be non-negative (can’t be less than zero).

In [ ]:


# Define the decision variables
model.power_transport_vehicles = Var(model.hours, within=NonNegativeReals, bounds=(0, 200))
model.set_point = Var(model.hours, within=NonNegativeIntegers, bounds=(18, 25))
model.num_active_chiller = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 10))
model.standby_power_down = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 1))
model.active_printers = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 3))
model.active_coffee_machines = Var(model.hours, within=NonNegativeIntegers, bounds=(0, 3))
model.num_servers = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_network_switches_poe = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_network_switches_non_poe = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_hdds = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))
model.num_ssds = Var(model.hours, within=NonNegativeIntegers, bounds=(1, 15))

# Define additional decision variables for constraint outcomes
model.chiller_energy = Var(model.hours, within=NonNegativeReals)
model.data_center_energy = Var(model.hours, within=NonNegativeReals)
model.production_energy = Var(model.hours, within=NonNegativeReals)
model.uta_energy = Var(model.hours, within=NonNegativeReals)
model.office_energy = Var(model.hours, within=NonNegativeReals)

# Display the model to check the variables
print(model)

**Define the constraints:**

In [ ]:
import joblib    

chiller_model = joblib.load('equation_modeling/models/chiller_consumption_model.joblib')
data_center_model = joblib.load('equation_modeling/models/data_center_consumption_model.joblib')
office_model = joblib.load('equation_modeling/models/office_consumption_model.joblib')
production_model = joblib.load('equation_modeling/models/production_consumption_model.joblib')
uta_model = joblib.load('equation_modeling/models/uta_consumption_model.joblib')

In [ ]:
chiller_intercept = chiller_model['intercept']
chiller_coefficients = chiller_model['coefficients']
chiller_features = chiller_model['features']

# Define the features as Pyomo expressions
def heat_index_set_point_diff_expr(model, h):
    return model.heat_index[h] - model.set_point[h]

# Define the chiller energy consumption constraint
def chiller_energy_constraint_rule(model, h):
    # Calculate the adjusted energy using the linear regression model
    COP_value = 3.0 
    
    adjusted_energy = ((
        chiller_intercept +
        chiller_coefficients[0] * heat_index_set_point_diff_expr(model, h) +
        chiller_coefficients[1] * model.efficiency_adjusted_power[h] +
        chiller_coefficients[2] * model.num_active_chiller[h]
    ) / (COP_value * model.efficiency_adjusted_power[h]))
    
    return model.chiller_energy[h] == adjusted_energy

# Add the constraint to the model
model.chiller_energy_constraint = Constraint(model.hours, rule=chiller_energy_constraint_rule)

In [ ]:
data_center_coefficients = data_center_model['coefficients']
data_center_features = data_center_model['features']

# Define the chiller energy consumption constraint
def data_center_constraint_rule(model, h):
    
    energy = (
        data_center_coefficients['server'] * model.num_servers[h] +
        data_center_coefficients['network_switch_poe'] * model.num_network_switches_poe[h] +
        data_center_coefficients['network_switch_non_poe'] * model.num_network_switches_non_poe[h] +
        data_center_coefficients['hdd'] * model.num_hdds[h] +
        data_center_coefficients['ssd'] * model.num_ssds[h]
    ) 
    
    return model.data_center_energy[h] == energy


# Add the constraint to the model
model.data_center_energy_constraint = Constraint(model.hours, rule=data_center_constraint_rule)

In [ ]:
production_intercept = production_model['intercept']
production_coefficients = production_model['coefficients']
production_features = production_model['features']

# Define the chiller energy consumption constraint
def production_energy_constraint_rule(model, h):
    
    energy = (
        production_intercept +
        production_coefficients[0] * model.power_transport_vehicles[h] +
        production_coefficients[1] * model.production_schedule[h] +
        production_coefficients[2] * model.maintenance_status[h] +
        production_coefficients[3] * model.volume_production_waste[h] +
        production_coefficients[4] * model.number_of_workers[h]
    ) 
    
    return model.production_energy[h] == energy

# Add the constraint to the model
model.production_energy_constraint = Constraint(model.hours, rule=production_energy_constraint_rule)

In [ ]:
uta_intercept = uta_model['intercept']
uta_coefficients = uta_model['coefficients']
uta_features = uta_model['features']

# Define the chiller energy consumption constraint
def uta_energy_constraint_rule(model, h):
    
    energy = (
        uta_intercept +
        uta_coefficients[0] * model.operational_presence[h] +
        uta_coefficients[1] * model.fabric_in_chamber[h] +
        uta_coefficients[2] * model.testing_schedule[h] +
        uta_coefficients[3] * model.workload[h] +
        uta_coefficients[4] * model.standby_power_down[h]
    ) 
    
    return model.uta_energy[h] == energy

# Add the constraint to the model
model.uta_energy_constraint = Constraint(model.hours, rule=uta_energy_constraint_rule)

In [ ]:
office_intercept = office_model['intercept']
office_coefficients = office_model['coefficients']
office_features = office_model['features']

# Define the chiller energy consumption constraint
def office_energy_constraint_rule(model, h):
    
    energy = (
        office_intercept +
        office_coefficients[0] * model.lightbulbs_active[h] +
        office_coefficients[1] * model.active_wall_plugs[h] +
        office_coefficients[2] * model.active_computers[h] +
        office_coefficients[3] * model.active_printers[h] +
        office_coefficients[4] * model.active_coffee_machines[h]
    ) 
    
    return model.office_energy[h] == energy

# Add the constraint to the model
model.office_energy_constraint = Constraint(model.hours, rule=office_energy_constraint_rule)

**Define Objective Function:**
This means we want to minimize the total cost

In [ ]:


# Define the objective function
def total_cost_rule(model):
    return sum(
        (model.price_mWh[h] / 1000) * (
            model.compressors_energy[h] +
            model.technological_centers_energy[h] +
            model.chiller_energy[h] +
            model.data_center_energy[h] +
            model.production_energy[h] +
            model.uta_energy[h] +
            model.office_energy[h]
        ) for h in model.hours
    )

# Add the objective function to the model
model.total_cost = Objective(rule=total_cost_rule, sense=minimize)

**Solve the optimization problem:**

In [ ]:
# Solve the model using GLPK
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Check the solver status
if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
    print("Solver found an optimal solution.")
else:
    print("Solver did not find an optimal solution. Status: ", results.solver.status)

# Display the objective function value
print("Total Cost: ", model.total_cost())

# Display the values of the decision variables
for h in model.hours:
    print(f"Hour {h}:")
    print(f"  Power Transport Vehicles: {model.power_transport_vehicles[h].value}")
    print(f"  Set Point: {model.set_point[h].value}")
    print(f"  Number of Active Chillers: {model.num_active_chiller[h].value}")
    print(f"  Standby Power Down: {model.standby_power_down[h].value}")
    print(f"  Active Printers: {model.active_printers[h].value}")
    print(f"  Active Coffee Machines: {model.active_coffee_machines[h].value}")
    print(f"  Number of Servers: {model.num_servers[h].value}")
    print(f"  Number of Network Switches (PoE): {model.num_network_switches_poe[h].value}")
    print(f"  Number of Network Switches (Non-PoE): {model.num_network_switches_non_poe[h].value}")
    print(f"  Number of HDDs: {model.num_hdds[h].value}")
    print(f"  Number of SSDs: {model.num_ssds[h].value}")
    print(f"  Chiller Energy: {model.chiller_energy[h].value}")
    print(f"  Data Center Energy: {model.data_center_energy[h].value}")
    print(f"  Production Energy: {model.production_energy[h].value}")
    print(f"  UTA Energy: {model.uta_energy[h].value}")
    print(f"  Office Energy: {model.office_energy[h].value}")
    print(f"  Compressors Energy: {model.compressors_energy[h]}")
    print(f"  Technological Centers Energy: {model.technological_centers_energy[h]}")


# DO IT ALL

In [ ]:
data = pd.read_csv('optimizer/optimizer_input/parameters.csv')
data['Time'] = pd.to_datetime(data['Time'])

In [ ]:
from datetime import timedelta, datetime

# Create empty dataframes to store the results
hourly_results_df = pd.DataFrame()
daily_results_df = pd.DataFrame()

# Define the start and end dates for the iteration
start_date = pd.to_datetime('2023-06-01')
end_date = pd.to_datetime('2023-07-01')

# Iterate over each day
current_date = start_date

while current_date < end_date:
    # Define the 24-hour period
    start_datetime = current_date
    end_datetime = current_date + timedelta(days=1)

    # Filter the data for the current 24-hour period
    filtered_data = data[(data['Time'] >= start_datetime) & (data['Time'] < end_datetime)]
    filtered_data = filtered_data.sort_values(by='Time').reset_index(drop=True)

    # Initialize the model parameters with filtered data
    model.production_schedule = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['production_schedule'].to_dict())
    model.maintenance_status = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['maintenance_status'].to_dict())
    model.volume_production_waste = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['volume_production_waste'].to_dict())
    model.number_of_workers = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['number_of_workers'].to_dict())
    model.heat_index = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['heat_index'].to_dict())
    model.efficiency_adjusted_power = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['efficiency_adjusted_power'].to_dict())
    model.compressors_energy = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['compressor_consumption'].to_dict())
    model.operational_presence = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['operational_presence'].to_dict())
    model.fabric_in_chamber = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['fabric_in_chamber'].to_dict())
    model.testing_schedule = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['testing_schedule'].to_dict())
    model.workload = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['workload'].to_dict())
    model.technological_centers_energy = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['technological_centers_consumption'].to_dict())
    model.lightbulbs_active = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['lightbulbs_active'].to_dict())
    model.active_wall_plugs = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['active_wall_plugs'].to_dict())
    model.active_computers = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['active_computers'].to_dict())
    model.price_mWh = Param(model.hours, within=NonNegativeReals, initialize=filtered_data.set_index(filtered_data.index)['price_mWh'].to_dict())

    # Solve the model
    solver = SolverFactory('glpk')
    results = solver.solve(model, tee=True)

    # Check the solver status
    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print(f"Solver found an optimal solution for {current_date.date()}.")
    else:
        print(f"Solver did not find an optimal solution for {current_date.date()}. Status: {results.solver.status}")

    # Store hourly results
    hourly_data = {
        'Date': [],
        'Hour': [],
        'Power Transport Vehicles': [],
        'Set Point': [],
        'Number of Active Chillers': [],
        'Standby Power Down': [],
        'Active Printers': [],
        'Active Coffee Machines': [],
        'Number of Servers': [],
        'Number of Network Switches (PoE)': [],
        'Number of Network Switches (Non-PoE)': [],
        'Number of HDDs': [],
        'Number of SSDs': [],
        'Compressor Energy': [],
        'Technological Centers Energy': [],
        'Chiller Energy': [],
        'Data Center Energy': [],
        'Production Energy': [],
        'UTA Energy': [],
        'Office Energy': [],
        'Total Cost': []
    }
    for h in model.hours:
        hourly_data['Date'].append(current_date)
        hourly_data['Hour'].append(h)
        hourly_data['Power Transport Vehicles'].append(model.power_transport_vehicles[h].value)
        hourly_data['Set Point'].append(model.set_point[h].value)
        hourly_data['Number of Active Chillers'].append(model.num_active_chiller[h].value)
        hourly_data['Standby Power Down'].append(model.standby_power_down[h].value)
        hourly_data['Active Printers'].append(model.active_printers[h].value)
        hourly_data['Active Coffee Machines'].append(model.active_coffee_machines[h].value)
        hourly_data['Number of Servers'].append(model.num_servers[h].value)
        hourly_data['Number of Network Switches (PoE)'].append(model.num_network_switches_poe[h].value)
        hourly_data['Number of Network Switches (Non-PoE)'].append(model.num_network_switches_non_poe[h].value)
        hourly_data['Number of HDDs'].append(model.num_hdds[h].value)
        hourly_data['Number of SSDs'].append(model.num_ssds[h].value)
        hourly_data['Compressor Energy'].append(model.compressors_energy[h])
        hourly_data['Technological Centers Energy'].append(model.technological_centers_energy[h])
        hourly_data['Chiller Energy'].append(model.chiller_energy[h].value)
        hourly_data['Data Center Energy'].append(model.data_center_energy[h].value)
        hourly_data['Production Energy'].append(model.production_energy[h].value)
        hourly_data['UTA Energy'].append(model.uta_energy[h].value)
        hourly_data['Office Energy'].append(model.office_energy[h].value)
        hourly_data['Total Cost'].append(model.total_cost())

    hourly_results_df = pd.concat([hourly_results_df, pd.DataFrame(hourly_data)], ignore_index=True)

    # Store daily results
    daily_results_df = pd.concat([daily_results_df, pd.DataFrame({
        'Date': [current_date],
        'Total Cost': [model.total_cost()]
    })], ignore_index=True)

    # Move to the next day
    current_date += timedelta(days=1)


# Reset index of the dataframes
hourly_results_df.reset_index(drop=True, inplace=True)
daily_results_df.reset_index(drop=True, inplace=True)

# Get the current datetime in the format YYYYMMDD_HHMMSS
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save the files to the new directory with the current datetime
hourly_results_df.to_csv(f'optimizer/optimizer_output/{current_datetime}_optimized_hourly_output.csv', index=False)
daily_results_df.to_csv(f'optimizer/optimizer_output/{current_datetime}_optimized_daily_output.csv', index=False)

print(f'{current_datetime}_optimized_hourly_output.csv output file saved.')
print(f'{current_datetime}_optimized_daily_output.csv output file saved.')

In [ ]:
ground_truth_hourly = pd.read_csv('optimizer/ground_truth/ground_truth_hourly.csv')
ground_truth_daily = pd.read_csv('optimizer/ground_truth/ground_truth_daily.csv')

evaluate_hourly = pd.merge(hourly_results_df, ground_truth_hourly, left_on=['Date', ' ', 'Hour'], right_on='Time', how='left')
evaluate_daily = pd.merge(daily_results_df, ground_truth_daily, on='Time', how='left')

evaluate_hourly.to_csv(f'optimizer/optimizer_output/{current_datetime}_evaluate_hourly_output.csv', index=False)
evaluate_daily.to_csv(f'optimizer/optimizer_output/{current_datetime}_evaluate_daily_output.csv', index=False)

